In [146]:
import numpy as np

In [111]:
p = 0.4 
gamma = 0.9
target = 100

In [108]:
class gambler:
    def __init__(self, p, gamma, target):
        self.p = p 
        self.gamma = gamma
        self.v = np.zeros(target)
        self.policy = [0 for i in range(len(v))]
    
    def bellman_update(s, v, policy, gamma): 
        
    
    def stake(self, a,p):
        """ Bet money on the coin flip
        a: amount staked, p:probability of heads on coin"""
        return (a*2 if coin_flip(p) == 1 else 0)
    
    def update_game_condition(self, s): 
        """ What's the state of the game?
        s: amount of money"""
        if (s <= 0):            condition = 'bankrupt'
        elif (s >= target):     condition = 'win'
        else:                   condition = 'playing'    
        return (condition)
    
    def bellman_update(self, pi, reward_win, s_lose, s_win): 
        return (pi * (  # Bellman update
                (1-p)* (             gamma * v[s_lose]                ) + # lose 
                p    * (reward_win + gamma * v[min((target-1), s_win)])   # win
        ))
    
    def policy_evaluation(self,s,v,policy, p, gamma):
        possible_stakes = [i for i in range(s)]  # actions 
        tmp_sum = 0
        for a in possible_stakes: 
            s_lose = s - a
            s_win  = s + a
            reward_win = 1 if s_win >= target else 0  # reward_lose is always 0 
            pi = policy[s][a]  # chance of this action
            tmp_sum += bellman_update(pi, reward_win, s_lose, s_win))
    
    def reset_game(self): 
        condition = 'playing'        
        
    def get_value_s(s): 
        """Calculate the value of a state 
        Keep betting until we reach 0 or 100"""
        
        

In [130]:
# equiprobable random policy to start across legal moves
# at policy[0], everything is 0. (you're buggered at this state)
policy = [[round(1/stake,2) if i < stake else 0 for i in range(target)] 
          for stake in range(target)]

In [183]:
s = 70 
v = np.zeros(target)


In [184]:
for i in range(100):
    for s in range(target):
        v[s] = bellman_update(s,v,policy, p, gamma)

In [185]:
np.round(v,3)

array([ 0.   ,  0.   ,  0.   ,  0.001,  0.002,  0.003,  0.004,  0.006,
        0.008,  0.01 ,  0.013,  0.016,  0.018,  0.023,  0.025,  0.03 ,
        0.031,  0.038,  0.046,  0.045,  0.054,  0.062,  0.069,  0.061,
        0.069,  0.077,  0.086,  0.097,  0.109,  0.093,  0.105,  0.119,
        0.134,  0.15 ,  0.164,  0.174,  0.185,  0.197,  0.209,  0.222,
        0.236,  0.166,  0.176,  0.187,  0.198,  0.209,  0.221,  0.234,
        0.246,  0.26 ,  0.274,  0.304,  0.335,  0.365,  0.396,  0.427,
        0.458,  0.489,  0.52 ,  0.551,  0.582,  0.614,  0.645,  0.677,
        0.709,  0.741,  0.773,  0.4  ,  0.416,  0.431,  0.446,  0.462,
        0.477,  0.493,  0.508,  0.524,  0.539,  0.555,  0.57 ,  0.586,
        0.602,  0.617,  0.633,  0.649,  0.664,  0.68 ,  0.696,  0.712,
        0.728,  0.744,  0.76 ,  0.776,  0.792,  0.808,  0.824,  0.84 ,
        0.856,  0.872,  0.888,  0.904])

In [181]:
np.round(v,3)

array([ 0.   ,  0.   ,  0.   ,  0.001,  0.002,  0.003,  0.004,  0.006,
        0.008,  0.01 ,  0.013,  0.016,  0.018,  0.023,  0.025,  0.03 ,
        0.031,  0.038,  0.046,  0.045,  0.054,  0.062,  0.069,  0.061,
        0.069,  0.077,  0.086,  0.097,  0.109,  0.093,  0.105,  0.119,
        0.134,  0.15 ,  0.164,  0.174,  0.185,  0.197,  0.209,  0.222,
        0.236,  0.166,  0.176,  0.187,  0.198,  0.209,  0.221,  0.234,
        0.246,  0.26 ,  0.274,  0.304,  0.335,  0.365,  0.396,  0.427,
        0.458,  0.489,  0.52 ,  0.551,  0.582,  0.614,  0.645,  0.677,
        0.709,  0.741,  0.773,  0.4  ,  0.416,  0.431,  0.446,  0.462,
        0.477,  0.493,  0.508,  0.524,  0.539,  0.555,  0.57 ,  0.586,
        0.602,  0.617,  0.633,  0.649,  0.664,  0.68 ,  0.696,  0.712,
        0.728,  0.744,  0.76 ,  0.776,  0.792,  0.808,  0.824,  0.84 ,
        0.856,  0.872,  0.888,  0.904])

In [107]:
g1.update_game_condition(200)

'win'

In [85]:
q = g1.test()

0.4


0.4

In [ ]:
def calc_v_s(s, p)
"""calculate the value function for a state
s: starting capital, 
p: coin flip"""
    